In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams['axes.prop_cycle'] = plt.cycler(color = sns.color_palette('dark'))
plt.style.use('dark_background')

## Load data


In [2]:
# Downloasd dataset


# !gdown https://drive.google.com/file/d/1DNgciEZTQWuo_wq4qMGIV8z3VC1Onjjv/view?usp=share_link --fuzzy --output ../../data/nlp/attention_data/
# !gdown https://drive.google.com/file/d/1ePfUhb2XeicnybAMu0y2w3BhiWm4XY8t/view?usp=share_link --fuzzy --output ../../data/nlp/attention_data/
# !gdown https://drive.google.com/file/d/1FRn6Gn3ijmCWfhUI6QkGm4g4LeQ7Veg0/view?usp=share_link --fuzzy --output ../../data/nlp/attention_data/
# !gdown https://drive.google.com/file/d/1Fw3yivW_PSH-C5T_4-9bWshxPHxvxtAi/view?usp=share_link --fuzzy --output ../../data/nlp/attention_data/
# !gdown https://drive.google.com/file/d/1F9r26d7emj43EG6kcMfvOsu1rI8yxR4d/view?usp=share_link --fuzzy --output ../../data/nlp/attention_data/
# !gdown https://drive.google.com/file/d/1zSYr8akNxtCCR_TxVIFrel1k254k-vin/view?usp=share_link --fuzzy --output ../../data/nlp/attention_data/
# !gdown https://drive.google.com/file/d/1zSYr8akNxtCCR_TxVIFrel1k254k-vin/view?usp=share_link --fuzzy --output ../../data/nlp/attention_data/
# !gdown https://drive.google.com/file/d/1zSYr8akNxtCCR_TxVIFrel1k254k-vin/view?usp=share_link --fuzzy --output ../../data/nlp/attention_data/
# !gdown https://drive.google.com/file/d/1I0vjwcnwfaMZdwhS-oJjqUoihl9NxZ1g/view?usp=share_link --fuzzy --output ../../data/nlp/attention_data/
# !gdown https://drive.google.com/file/d/1tX9mZBStZGCQmOhgaCu-tSdlXqQU77ou/view?usp=share_link --fuzzy --output ../../data/nlp/attention_data/
# !gdown https://drive.google.com/file/d/1-GNcBpiB3Y0RzbJLawxL31IjkTBtpUz2/view?usp=share_link --fuzzy --output ../../data/nlp/attention_data/

In [3]:
def load_mams_data(path):
    df = pd.read_csv(path)
    df = df[['text' , 'term' , 'polarity']]
    return df

In [4]:
!ls ../../data/nlp/attention_data/

laptop_dev.csv   mams_test.csv    rest16_dev.csv
laptop_test.csv  mams_train.csv   rest16_test.csv
laptop_train.csv mams_val.csv     rest16_train.csv


In [5]:
import os
basedir = '../../data/nlp/attention_data'
# list file names starting with mams:
files = os.listdir(f'{basedir}')
mams_files = [f"{basedir}/{file}" for file in files if file.startswith('mams')]
mamsval , mamstest , mamstrain = list(map(load_mams_data , mams_files))
mamstrain.shape , mamstest.shape , mamsval.shape

((11186, 3), (1336, 3), (1332, 3))

In [6]:
# load laptop data
def load_laptop_data(path):
    textds = []
    termds = []
    polarityds = []

    with open(path , 'r') as file:
        data = file.read().splitlines()


    terms = []
    polarities = []
    text = ''
    polarity = ''
    term = ''

    for row in data[2:]:
        if len(row) == 1:
            if term != '':
                terms.append(term)
                polarities.append(polarity)
            for i in range(len(terms)):
                textds.append(text.strip())
                polarityds.append(polarities[i])
                termds.append(terms[i].strip())
            
            text = ''
            polarity = ''
            term = ''
            terms = []
            polarities = []

        else:
            token , label = row.split(',' , 1)
            text += ' ' + token

            if label == 'T-POS':
                label = 'positive'
            elif label == 'T-NEG':
                label = 'negative'
            elif label == 'T-NEU':
                label = 'neutral'
            else:
                if term != '':
                    terms.append(term)
                    polarities.append(polarity)

                term = ''
                polarity = ''
                continue

            if polarity == label:
                term += ' ' + token

            else:
                if term != '':
                    terms.append(term)
                    polarities.append(polarity)

                polarity = label
                term = token

    df = pd.DataFrame()
    df['text'] = textds
    df['term'] = termds
    df['polarity'] = polarityds


    return df

laptop_files = [f'{basedir}/{file}' for file in files if file.startswith('laptop')]
lapval , laptest , laptrain = list(map(load_laptop_data , laptop_files))
laptrain.shape , laptest.shape , lapval.shape

((1916, 3), (577, 3), (246, 3))

In [7]:
# load restaurant data:
restFiles = [f'{basedir}/{file}' for file in files if file.startswith('rest')]
restval , resttrain , resttest = list(map(load_laptop_data , restFiles))
resttrain.shape , resttest.shape , restval.shape

((1501, 3), (585, 3), (159, 3))

In [8]:
traindf = pd.concat([mamstrain , laptrain , resttrain , mamstest , resttest , laptest]).sample(frac = 1).reset_index(drop = True)
valdf = pd.concat([mamsval , lapval , restval]).sample(frac = 1).reset_index(drop = True)

In [9]:
print('*'*30 , 'data' , '*' *30)
traindf.head()

****************************** data ******************************


,text,term,polarity
0,"Though the picture "" video "" and music softwar...",picture,negative
1,We asked for the manager who told us that it w...,manager,positive
2,Our server (after 4 reminders) finally got aro...,meal,neutral
3,"The food and beer make up for it, but it would...",beer,positive
4,"One more tip "" please purchase this model and ...",4GB stick of RAM,positive


In [10]:
print('*'*30 , 'data uniqueness' , '*' *30)
traindf.nunique()

****************************** data uniqueness ******************************


text        7838
term        4488
polarity       3
dtype: int64

In [11]:
print('*'*30 , 'Sentiment labels' , '*' *30)
traindf.polarity.value_counts()

****************************** Sentiment labels ******************************


polarity
positive    6456
neutral     6270
negative    4375
Name: count, dtype: int64

In [12]:
#consider only positive and negative polarities and convert them to int indexes:
traindf = traindf[traindf['polarity'] != 'neutral']
valdf = valdf[valdf['polarity'] != 'neutral']

pol2id = {'positive' : 1 , 'negative' : 0}

for data in [traindf , valdf]:
    data['polarity'] = data['polarity'].apply(lambda x: pol2id[x])
    data['text'] = data['text'].apply(lambda x : x.lower())
    data['term'] = data['term'].apply(lambda x : x.lower())

    data = data.drop_duplicates(['text' , 'term' , 'polarity'])
traindf.head()

,text,term,polarity
0,"though the picture "" video "" and music softwar...",picture,0
1,we asked for the manager who told us that it w...,manager,1
3,"the food and beer make up for it, but it would...",beer,1
4,"one more tip "" please purchase this model and ...",4gb stick of ram,1
5,"calle ocho is definitely high on my list, the ...",mojitos,1


## Tokenize and embedding and train-test split:


In [13]:
#load embedding model and convert to embeddings:
import gensim.downloader as api

embedding_size = 300

word2vec = api.load('word2vec-google-news-300')

#tokenize the text

tokenizer = tf.keras.preprocessing.text.Tokenizer(lower = True)
tokenizer.fit_on_texts(traindf['text'])

In [14]:
# create embedding matrix:
ttokens = len(tokenizer.word_index) + 1
embeddings = np.zeros((ttokens , embedding_size))

hits = 0
misses = 0

for word , i in tokenizer.word_index.items():
    embedding = None
    try:
        embedding = word2vec[word]
    except Exception:
        pass

    if embedding is not None:
        embeddings[i] = embedding
        hits+=1
    else: misses += 1

del word2vec
print(f"converted {hits} words and missed {misses} words")

converted 9019 words and missed 1234 words


In [15]:
#tokenize and pad:
def toknpad(text , seqlen):
    tokens = tokenizer.texts_to_sequences([text])
    paddedTokens = tf.keras.preprocessing.sequence.pad_sequences(tokens , maxlen = seqlen)
    return paddedTokens[0]

def prepare_tokenize(dataset):
    texts = []
    terms = []
    polarities = []

    for id , row in dataset.iterrows():
        texts.append(toknpad(row['text'] , 128))
        terms.append(toknpad(row['term'] , 4))
        polarities.append(row['polarity'])

    texts = tf.constant(texts)
    terms = tf.constant(terms)
    polarities = tf.constant(polarities)

    return (texts , terms , polarities)

ttexts , tterms , tpolarities = prepare_tokenize(traindf)
vtexts , vterms , vpolarities = prepare_tokenize(valdf)

In [16]:
print("*"*64)
print("Review text", ttexts[0])
print("*"*64)
print("Aspect text", tterms[0])
print("*"*64)
print("Polarity", tpolarities[0])
print("*"*64)

****************************************************************
Review text tf.Tensor(
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0  117    1 2059  833    2  278  321    7 1704  574    4 1096 2224
  321 2225  384    4 3429   13  210    1 1097    9 3430    2   55 2992
 1534 3431], shape=(128,), dtype=int32)
****************************************************************
Aspect text tf.Tensor([   0    0    0 2059], shape=(4,), dtype=int32)
****************************************************************
Polarity tf.Tensor(0, shape=(), d

## model implementation:


In [186]:
class Attention(tf.keras.layers.Layer):
    def __init__(self, units, name="simple_attention"):
        super(Attention, self).__init__(name=name)
        self.units = units
        self.Wk = tf.keras.layers.Dense(units)
        self.Wq = tf.keras.layers.Dense(units)
        self.Wv = tf.keras.layers.Dense(units)

    def call(self, keys, queries):
        """
        keys:    (batch, R, dim)  -> review encoder output
        queries: (batch, A, dim)  -> aspect encoder output
        """

        # Linear projections
        K = self.Wk(keys)       # (batch, R, units)
        Q = self.Wq(queries)    # (batch, A, units)
        V = self.Wv(keys)       # (batch, R, units)

        # Compute raw scores
        scores = tf.matmul(K , Q , transpose_b=True)   # (batch, k , q)

        # Softmax over key dimension (review length R)
        attn = tf.nn.softmax(scores, axis=-1)        # (batch, l , q)

        print(f"{attn.shape=}")
        # Weighted sum to get context


        attnexp = tf.expand_dims(attn , axis = -1)
        vexp = tf.expand_dims(V , axis = 2)

        print(f"{attnexp.shape=} , {vexp.shape=}")

        context = attnexp * vexp
        
        print(f"{context.shape=}")
        out = tf.reduce_sum(context , axis = [2,3] , keepdims = True)
        out = tf.squeeze(out , axis = [2,3])


        # If aspect_len = 1 → squeeze to (batch, units)
        # if context.shape[1] == 1:
        #     context = tf.squeeze(context, axis=1)    # (batch, units)
        #     attn = tf.squeeze(attn, axis=1)          # (batch, R)


        print(f"{out.shape=}")
        return out, attn


In [187]:
#create a functional model:

review_input = tf.keras.layers.Input(shape = (128 , ) , dtype = 'int32' , name = 'review_input')
aspect_input = tf.keras.layers.Input(shape = (4 ,) , dtype = 'int32' , name = 'aspect_input')

#get embeddings (same embedding matrix for review and aspect):

embedding_layer = tf.keras.layers.Embedding(input_dim = ttokens , output_dim= embedding_size , embeddings_initializer= tf.keras.initializers.Constant(embeddings) , mask_zero = True , name = 'shared_embedding_layer')

rembeddings = embedding_layer(review_input)
aembeddings = embedding_layer(aspect_input)

rmask = embedding_layer.compute_mask(review_input)
amask = embedding_layer.compute_mask(aspect_input)


#get key , query and value vectors:

rencoder = tf.keras.layers.GRU(256 , return_sequences= True , name = 'review_encoder')
aencoder = tf.keras.layers.GRU(256 , return_sequences= True , name = 'aspect_encoder')


rkeys = rencoder(rembeddings)
akeys = aencoder(aembeddings)

#get attention vector:

attention_layer = Attention(256)

values , attention_weights = attention_layer(rkeys , akeys)

print(f"{values.shape=}")
#compute sentiment:

values = tf.keras.layers.Dense(64 , activation = 'relu' , name = 'penultimate_layer')(values)
output = tf.keras.layers.Dense(1 , activation = 'sigmoid' , name = 'final_output')(values)
print(f"{output.shape=}")

#create model:

model = tf.keras.Model(inputs = [review_input , aspect_input] , outputs = output , name = 'sentiment_model_with_attention')
# model.summary()


#compile model:
learning_rate = tf.keras.optimizers.schedules.PolynomialDecay(1e-2, 236, 1e-4)

model.compile(
    optimizer=tf.optimizers.Adam(learning_rate),
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)


#train:
history = model.fit(
    x = [ttexts , tterms],
    y = tpolarities,
    batch_size = 32,
    epochs = 5,
)


attn.shape=TensorShape([None, 128, 4])
attnexp.shape=TensorShape([None, 128, 4, 1]) , vexp.shape=TensorShape([None, 128, 1, 256])
context.shape=TensorShape([None, 128, 4, 256])
out.shape=TensorShape([None, 128])
attn.shape=TensorShape([None, 128, 4])
attnexp.shape=TensorShape([None, 128, 4, 1]) , vexp.shape=TensorShape([None, 128, 1, 256])
context.shape=TensorShape([None, 128, 4, 256])
out.shape=TensorShape([None, 128])
values.shape=(None, 128)
output.shape=(None, 1)
Epoch 1/5
attn.shape=TensorShape([None, 128, 4])
attnexp.shape=TensorShape([None, 128, 4, 1]) , vexp.shape=TensorShape([None, 128, 1, 256])
context.shape=TensorShape([None, 128, 4, 256])
out.shape=TensorShape([None, 128])


/opt/anaconda3/envs/tensorflow/lib/python3.11/site-packages/keras/src/layers/layer.py:965: UserWarning: Layer 'simple_attention' (of type Attention) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


attn.shape=TensorShape([None, 128, 4])
attnexp.shape=TensorShape([None, 128, 4, 1]) , vexp.shape=TensorShape([None, 128, 1, 256])
context.shape=TensorShape([None, 128, 4, 256])
out.shape=TensorShape([None, 128])
339/339 ━━━━━━━━━━━━━━━━━━━━ 73s 210ms/step - accuracy: 0.5971 - loss: 1.6470
Epoch 2/5
339/339 ━━━━━━━━━━━━━━━━━━━━ 77s 228ms/step - accuracy: 0.7796 - loss: 0.4568
Epoch 3/5
339/339 ━━━━━━━━━━━━━━━━━━━━ 80s 235ms/step - accuracy: 0.8002 - loss: 0.4220
Epoch 4/5
339/339 ━━━━━━━━━━━━━━━━━━━━ 80s 236ms/step - accuracy: 0.8192 - loss: 0.3886
Epoch 5/5
339/339 ━━━━━━━━━━━━━━━━━━━━ 84s 247ms/step - accuracy: 0.8286 - loss: 0.3698


In [84]:
# ensure shapes are (N,1) and float32
tpolarities = np.asarray(tpolarities).reshape(-1,1).astype('float32')
vpolarities = np.asarray(vpolarities).reshape(-1,1).astype('float32')
print("after reshape:", tpolarities.shape, vpolarities.shape)


after reshape: (10831, 1) (1084, 1)


In [80]:
ttexts.shape , tterms.shape , tpolarities.shape

(TensorShape([10831, 128]), TensorShape([10831, 4]), (10831, 1))

In [85]:
history = model.fit(
    x = [ttexts , tterms],
    y = tpolarities,
    batch_size = 32,
    epochs = 5,
)

Epoch 1/5
attention_computation begin.
keymask.shape=TensorShape([None, 128, 1]) , querymask.shape=TensorShape([None, 1, 4]) , scores.shape=TensorShape([None, 128, 4])
attention_computation done.
cvector.shape=TensorShape([None, 128])
attention_computation begin.
keymask.shape=TensorShape([None, 128, 1]) , querymask.shape=TensorShape([None, 1, 4]) , scores.shape=TensorShape([None, 128, 4])
attention_computation done.
cvector.shape=TensorShape([None, 128])


InvalidArgumentError: Graph execution error:

Detected at node gradient_tape/compile_loss/binary_crossentropy/mul_1/BroadcastGradientArgs defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/opt/anaconda3/envs/tensorflow/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/opt/anaconda3/envs/tensorflow/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/opt/anaconda3/envs/tensorflow/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/opt/anaconda3/envs/tensorflow/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/opt/anaconda3/envs/tensorflow/lib/python3.11/asyncio/base_events.py", line 608, in run_forever

  File "/opt/anaconda3/envs/tensorflow/lib/python3.11/asyncio/base_events.py", line 1936, in _run_once

  File "/opt/anaconda3/envs/tens

In [33]:
# if using numpy arrays / tf.data
print(ttexts.shape, tterms.shape, tpolarities.shape)   # shapes you pass as x,y
# or if validation_data tuple:
print(vtexts.shape, vterms.shape, vpolarities.shape)


(10831, 128) (10831, 4) (10831, 1)
(1084, 128) (1084, 4) (1084, 1)


In [32]:
# ensure shapes are (N,1) and float32
tpolarities = np.asarray(tpolarities).reshape(-1,1).astype('float32')
vpolarities = np.asarray(vpolarities).reshape(-1,1).astype('float32')
print("after reshape:", tpolarities.shape, vpolarities.shape)


after reshape: (10831, 1) (1084, 1)
